In [127]:
import numpy as np

In [128]:
import pandas as pd

# csv file already contains index column so we don't need to specify index_col
X_train = pd.read_csv('dataset/train_data.csv', index_col=0)

In [129]:
X_train.head()

,string,valid
0,BPTVXPVXPTTVXVE,0
1,BPVXPVXPVXPTTVXPTTVSE,0
2,BNAFUCCAFUCCAFUADE,0
3,BNWWWWWVZUWWVZRE,0
4,BPTVPXVPXTVPXTTTTTTVPSE,1


In [130]:
# only select the rows with valid = 1
X_train = X_train[X_train['valid'] == 1]

# drop the valid column
X_train = X_train.drop('valid', axis=1)

In [131]:
X_train.head()

,string
4,BPTVPXVPXTVPXTTTTTTVPSE
5,BTXXTTVPXVPXVVE
6,BPTTTTTTVPXVPXVPXVVE
7,BTSXXTTTTTVPXTTTVPXVPSE
10,BTXXVPXTTVPXTTTTVPXVPXTTVVE


In [132]:
# reset the index
X_train = X_train.reset_index(drop=True)

In [133]:
X_train.head()

,string
0,BPTVPXVPXTVPXTTTTTTVPSE
1,BTXXTTVPXVPXVVE
2,BPTTTTTTVPXVPXVPXVVE
3,BTSXXTTTTTVPXTTTVPXVPSE
4,BTXXVPXTTVPXTTTTVPXVPXTTVVE


In [139]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode(characters, matrix):
    # convert each element in the matrix to a list of characters
    matrix = [[char for char in row] for row in matrix]
    # Create a dictionary of characters
    char_dict = {char: i for i, char in enumerate(characters)}
    # Create a list of lists of character indices
    char_indices = [[char_dict[char] for char in row] for row in matrix]
    # Create a OneHotEncoder object
    encoder = OneHotEncoder()
    # Fit the encoder on the character indices
    encoder.fit(char_indices)
    # Transform the character indices into a one-hot encoded matrix
    one_hot_matrix = encoder.transform(char_indices).toarray()
    return one_hot_matrix

# Example usage
characters = ['a', 'b', 'c']
matrix = ['abc', 'bca', 'cab']
one_hot_matrix = one_hot_encode(characters, matrix)
print(one_hot_matrix)


[[1. 0. 0. 0. 1. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 1. 0.]]


In [ ]:
# import one hot encoder
from sklearn.preprocessing import OneHotEncoder

# reber grammar vocabulary
vocabulary = np.array(['B', 'T', 'P', 'S', 'X', 'V', 'E'])

# create one hot encoder object
one_hot_encoder = OneHotEncoder(categories=[vocabulary])

# encode X_train data according to the vocabulary
